<a href="https://colab.research.google.com/github/gyuwon0112/MathBox/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install beautifulsoup4
#!pip install -U selenium
#!apt install chromium-chromedriver
#!pip install webdriver-manager
!pip install kora -q

from kora.selenium import wd
from bs4 import BeautifulSoup as bs
from time import sleep
import random
import urllib.request as ur
import pandas as pd

Course = []
Duration = []
Start_Date = []
Offered_By = []
No_Of_Reviews = []
rat = []
t = []

m = 1# page 수

while m < (subject/15 + 1): # page까지의 정보 스크랩
    if m == 1 : # 첫 page의 url
        url = 'https://www.classcentral.com/subject/data-science' 
    else : # 두번째 page부터는 뒤를 수정해서 page만 넣어주면 됨
        url = 'https://www.classcentral.com/subject/data-science?page=%d'%m
    
    #print(url)

    html = bs(ur.urlopen(url).read(), "html.parser")
    
    
    #wd.get(url)
    #html = BeautifulSoup(wd.page_source)
    #html = bs(ur.urlopen(url).read(), 'html.parser')


    def find_1st(string, substring):
        return string.find(substring, string.find(substring))
        
    def find_2nd(string, substring):
        return string.find(substring, string.find(substring) + 1)

    # Course
    for i in html.findAll("h2", {"class" : "text-1 weight-semi line-tight margin-bottom-xxsmall"}):
        b = str(i)
        #print(b[find_1st(b,'>')+1:find_2nd(b,'<')])
        Course.append(b[find_1st(b,'>')+1:find_2nd(b,'<')])
          

    #print(Course)
    course = []
    for i in Course:
        i = i.strip() # The strip() method removes any leading and trailing characters
                    # (space is the default leading character to remove)
        #print(i)
        course.append(i)
        

    #print(course)

    # Provider/Offered By:
    for i in html.findAll('a',href=True, attrs={'class':'hover-underline color-charcoal text-3 margin-left-small line-tight'}):
        b = str(i)
        #print(b[find_1st(b,'>')+1:find_2nd(b,'<')])
        Offered_By.append(b[find_1st(b,'>')+1:find_2nd(b,'<')])
    
    provider = []
    for i in Offered_By:
        i = i.strip()
        provider.append(i)

    #print(provider)

    # Ratings
    for d in html.findAll('span', attrs={'role':'img', 'class':'cmpt-rating-medium '}):
        rat.append(d.get('aria-label'))

    Rating = []
    for i in rat:
        i = i.strip()
        #print(i)
        Rating.append(i)



    #print(Rating)

    # Num of Reviews
    for i in html.findAll("span", attrs={'class':'text-3 color-gray margin-left-xxsmall'}):
        b = str(i)
        if i is not None :
            No_Of_Reviews.append(i.text)
        else:
            No_Of_Reviews.append('-1')
            
        #print(b[find_1st(b,'>')+1:find_2nd(b,'<')])
        #No_Of_Reviews.append(b[find_1st(b,'>')+1:find_2nd(b,'<')])
    num_reviews = []
    for i in No_Of_Reviews:
        i = i.strip()
        #print(i)
        num_reviews.append(i)
        
    #print(num_reviews)

    for d in html.findAll('li', attrs={'class':'nowrap padding-xsmall border-top border-gray-light row horz-align-left'}):
        abc = d.find('span', attrs={'aria-label':'Workload and duration'})
        if abc is not None:
            #print(abc.text)
            t.append(abc.text)
            
    duration = []
    for i in t:
        i = i.strip()
        #print(i)
        duration.append(i)

    m = m + 1 # 페이지 넘기기

    if(len(num_reviews)!=len(course)):
        for p in range(len(course)-len(num_reviews)) :
            num_reviews.append(-1)
    
    #print(len(course), len(Rating), len(num_reviews), len(provider), len(duration))
    


# 원하는 페이지까지 스크롤링한 후 출력한다
dfDS = pd.DataFrame({'course':course, 'ratings':Rating,'No_of_Reviews':num_reviews,'provider':provider, 'Duration':duration})
dfDS


##3. 본 데이터로부터 얻을 수 있는 정보

* 본 프로젝트에서는 훌륭한 데이터 과학 교육과정을 찾기 위해 [Class Central](https://www.classcentral.com/subject/data-science)이라고 불리는 홈페이지를  스크랩 한 것이다.

* 이 홈페이지에는 '컴퓨터과학', '비지니스', '엔지니어', '교육' 등 다양한 토픽의 온라인 교육과정을 보여주고 있다. 

* 본 프로젝트에서는 '데이터과학'과 관련된 정보를 스크랩하였다.

* 스크랩한 데이터에는 '강의명', '제공자', '평가', '리뷰', '기간'의 정보가 포함되어 있다.
    * '리뷰' : 리뷰 수가 아닌 '-1'로 표기된 강의들은 리뷰가 등록되지 않은 신설 강의에 해당한다
    * '기간' : 주당 지정된 시간만큼 학습 할 경우, 몇 주만에 끝낼 수 있는 지에 대한 정보를 제공한다.